# Hull Tactical - Market Prediction Challenge

## Objective
Build a model that predicts excess returns and includes a betting strategy designed to outperform the S&P 500 while staying within a 120% volatility constraint.

## Key Challenge
- Predict optimal allocation to S&P 500 (0 to 2 range, allowing leverage)
- Maximize modified Sharpe ratio
- Use time-series cross-validation to prevent look-ahead bias
- Challenge the Efficient Market Hypothesis (EMH)

## Approach
1. Feature Engineering: Technical indicators, momentum, volatility, sentiment
2. Multiple Models: Ensemble of XGBoost, LightGBM, and Neural Networks
3. Position Sizing: Dynamic allocation with risk management
4. Evaluation: Custom Sharpe metric with volatility penalty

## 1. Import Required Libraries

In [35]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Gradient Boosting
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Technical Analysis - Install if needed
try:
    import talib
    TALIB_AVAILABLE = True
except ImportError:
    print("⚠ TA-Lib not installed. Installing via pip...")
    print("Note: TA-Lib requires binary dependencies. If pip install fails,")
    print("please install from: https://github.com/mrjbq7/ta-lib")
    TALIB_AVAILABLE = False

from scipy import stats
from scipy.optimize import minimize

# Utilities
from datetime import datetime, timedelta
import os
import json
from pathlib import Path

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully")
if not TALIB_AVAILABLE:
    print("⚠ TA-Lib not available - will use alternative implementations")

⚠ TA-Lib not installed. Installing via pip...
Note: TA-Lib requires binary dependencies. If pip install fails,
please install from: https://github.com/mrjbq7/ta-lib
✓ All libraries imported successfully
⚠ TA-Lib not available - will use alternative implementations


## 2. Configuration and Data Loading

In [36]:
def load_data():
    """
    Load training and test data.
    Assumes data files: train.csv, test.csv, or similar
    Adjust based on actual data format provided by competition
    """
    try:
        # Try to load from competition data directory
        train_df = pd.read_csv(os.path.join(CONFIG['data_path'], 'train.csv'))
        print(f"Training data loaded: {train_df.shape}")
        
        # Check if test data exists
        test_path = os.path.join(CONFIG['data_path'], 'test.csv')
        if os.path.exists(test_path):
            test_df = pd.read_csv(test_path)
            print(f"Test data loaded: {test_df.shape}")
        else:
            test_df = None
            print("No test file found (will use API for submission)")
        
        return train_df, test_df
    
    except FileNotFoundError:
        print("⚠ Data files not found. Please ensure data is in the correct location.")
        print(f"Looking in: {CONFIG['data_path']}")
        print("\nCreating sample data for demonstration...")
        
        # Create sample data for development
        dates = pd.date_range(start='2010-01-01', end='2024-12-31', freq='D')
        n = len(dates)
        
        # Simulate S&P 500 returns (random walk with drift)
        np.random.seed(42)
        returns = np.random.normal(0.0003, 0.01, n)  # ~7.5% annual return, 16% volatility
        
        train_df = pd.DataFrame({
            'Date': dates,
            'SPY_Close': 100 * np.exp(np.cumsum(returns)),
            'SPY_Volume': np.random.uniform(50e6, 150e6, n),
            'VIX_Close': np.maximum(10, 20 + np.random.normal(0, 5, n)),
            'DXY_Close': 95 + np.random.normal(0, 2, n),
            'TNX_Close': np.maximum(0.5, 2.5 + np.random.normal(0, 0.5, n)),
        })
        
        # Calculate returns
        train_df['SPY_Return'] = train_df['SPY_Close'].pct_change()
        
        # Add target (next day return)
        train_df['Target'] = train_df['SPY_Return'].shift(-1)
        
        print(f"Sample data created: {train_df.shape}")
        return train_df, None

# Load data
train_data, test_data = load_data()

# Display first few rows
if train_data is not None:
    print("\nData Preview:")
    display(train_data.head())
    print("\nData Info:")
    print(train_data.info())
    print("\nBasic Statistics:")
    display(train_data.describe())

Training data loaded: (9021, 98)
Test data loaded: (10, 99)

Data Preview:


,date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,I8,I9,M1,M10,M11,M12,M13,M14,M15,M16,M17,M18,M2,M3,M4,M5,M6,M7,M8,M9,P1,P10,P11,P12,P13,P2,P3,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
0,0,0,0,0,1,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002421,0.000301,-0.003038
1,1,0,0,0,1,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008495,0.000303,-0.009114
2,2,0,0,0,1,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009624,0.000301,-0.010243
3,3,0,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004662,0.000299,0.004046
4,4,0,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011686,0.000299,-0.012301



Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9021 entries, 0 to 9020
Data columns (total 98 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date_id                        9021 non-null   int64  
 1   D1                             9021 non-null   int64  
 2   D2                             9021 non-null   int64  
 3   D3                             9021 non-null   int64  
 4   D4                             9021 non-null   int64  
 5   D5                             9021 non-null   int64  
 6   D6                             9021 non-null   int64  
 7   D7                             9021 non-null   int64  
 8   D8                             9021 non-null   int64  
 9   D9                             9021 non-null   int64  
 10  E1                             7237 non-null   float64
 11  E10                            8015 non-null   float64
 12  E11                            8015 

,date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,I8,I9,M1,M10,M11,M12,M13,M14,M15,M16,M17,M18,M2,M3,M4,M5,M6,M7,M8,M9,P1,P10,P11,P12,P13,P2,P3,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
count,9021.000000,9021.000000,9021.000000,9021.000000,9021.000000,9021.000000,9021.000000,9021.000000,9021.000000,9021.000000,7237.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,7405.000000,8015.000000,8015.000000,8015.000000,8015.000000,2052.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,3474.000000,8015.000000,8015.000000,8015.000000,3481.000000,3481.000000,8015.000000,8015.000000,8015.000000,8015.000000,5804.000000,7003.000000,8015.000000,5738.000000,3978.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,7447.000000,7383.000000,7405.000000,8015.000000,8015.000000,8015.000000,8015.000000,8015.000000,5484.000000,8015.000000,3288.000000,8015.000000,7510.000000,8015.000000,8015.000000,6012.000000,8015.000000,8015.000000,2972.000000,8015.000000,8015.000000,7510.000000,8015.000000,8015.000000,8015.000000,7509.000000,8015.000000,7510.000000,8015.000000,4482.000000,9021.000000,9021.000000,9021.000000
mean,4510.000000,0.031593,0.031593,0.047777,0.573994,0.190445,-0.238111,0.045671,0.142667,0.143000,1.564376,0.503676,0.125415,0.118313,0.012279,0.006991,0.486758,-0.047978,-0.009339,0.098024,0.116761,0.504014,0.905390,0.346242,0.016857,0.598541,0.122023,-0.034635,-0.272228,0.265910,0.745320,-0.522922,0.607538,0.551064,0.186341,0.456389,0.706785,0.572773,0.187819,-0.620970,0.058347,-0.288699,0.478585,-0.951588,-0.863647,0.461418,0.231891,0.272509,0.599215,0.085883,0.149234,-0.002284,0.236473,0.231195,0.002751,0.436222,0.382007,0.527662,1.468845,1.260329,-0.020481,0.508742,-0.364950,0.495414,0.500406,0.001178,0.052918,0.249174,1.543061,0.393674,0.241018,0.437959,0.432863,0.265411,0.025082,0.060300,0.456036,0.034151,0.515287,0.494731,0.079207,0.456539,0.325772,-0.003620,0.230551,0.253973,0.111299,0.508960,0.489076,0.506589,0.373584,0.288874,0.145886,0.303203,0.125155,0.000471,0.000107,0.000053
std,2604.282723,0.174923,0.174923,0.213307,0.494522,0.392674,0.425951,0.208783,0.349752,0.350092,0.632544,0.336882,0.245352,0.251567,0.019234,0.012076,0.349147,1.135656,1.157585,1.144661,1.245992,1.422951,1.270370,1.506513,0.041801,0.337284,0.221698,1.910978,1.511888,0.301335,0.245415,1.252119,0.331993,0.306722,1.639333,0.302416,0.257393,0.325655,1.638840,0.996912,1.316043,1.221487,1.394923,0.651200,0.192252,0.273803,0.326578,0.226543,0.344105,1.091028,1.220819,1.092406,1.549889,1.359659,1.018703,0.317700,1.409160,0.327368,0.813670,1.095447,1.085142,0.283680,1.444433,0.288427,0.288163,1.068961,1.150434,1.132135,0.707622,0.385385,1.420260,0.322434,0.324621,0.956397,1.022805,1.006811,0.326317,1.134158,0.288787,0.307013,1.097029,0.328112,0.345797,1.241267,0.315737,0.306309,1.328520,0.305945,0.306060,0.306216,1.151136,0.312905,1.324779,0.350627,1.273912,0.010541,0.000088,0.010558
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.325149,0.000661,0.000661,0.000661,0.000661,0.000661,0.000661,-5.130519,-4.302885,-4.131097,-1.826114,-4.675791,-3.308764,-5.374951,0.000661,0.000661,0.000661,-19.918972,-2.457316,0.000661,0.002646,-3.542308,0.000661,0.000661,-4.449235,0.000661,0.003307,0.000661,-4.546619,-2.522870,-4.765828,-3.190249,-2.993367,-1.932110,-1.325950,0.000661,0.000661,0.000661,0.000661,-2.667421,-1.802507,-9.389925,-3.424095,-2.469748,-4.317569,0.000661,-2.831429,0.000661,-1.162766,-2.719004,-1.539300,0.002315,-3.140889,0.043981,0.066138,-1.430121,-0.780115,-3

## 3. Feature Engineering

Creating comprehensive features to capture market patterns:
- **Technical Indicators**: Moving averages, RSI, MACD, Bollinger Bands
- **Momentum Features**: Rate of change, price momentum
- **Volatility Features**: Historical volatility, ATR, Bollinger width
- **Market Regime**: Trend strength, market state classification
- **Sentiment Proxies**: VIX, put/call ratios (if available)
- **Macro Indicators**: Interest rates, dollar index

In [37]:
class FeatureEngineer:
    """
    Comprehensive feature engineering for market prediction
    """
    
    def __init__(self, lookback_periods=[5, 10, 20, 50, 100, 200]):
        self.lookback_periods = lookback_periods
        
    def _calculate_rsi(self, prices, period=14):
        """Calculate RSI manually if talib not available"""
        delta = pd.Series(prices).diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi.values
    
    def _calculate_macd(self, prices, fast=12, slow=26, signal=9):
        """Calculate MACD manually if talib not available"""
        prices_series = pd.Series(prices)
        ema_fast = prices_series.ewm(span=fast, adjust=False).mean()
        ema_slow = prices_series.ewm(span=slow, adjust=False).mean()
        macd = ema_fast - ema_slow
        macd_signal = macd.ewm(span=signal, adjust=False).mean()
        macd_hist = macd - macd_signal
        return macd.values, macd_signal.values, macd_hist.values
    
    def _calculate_bbands(self, prices, period=20, std_dev=2):
        """Calculate Bollinger Bands manually if talib not available"""
        prices_series = pd.Series(prices)
        middle = prices_series.rolling(window=period).mean()
        std = prices_series.rolling(window=period).std()
        upper = middle + (std * std_dev)
        lower = middle - (std * std_dev)
        return upper.values, middle.values, lower.values
    
    def add_returns_features(self, df, price_col='SPY_Close'):
        """Calculate returns at various timeframes"""
        df = df.copy()
        
        # Simple returns
        for period in self.lookback_periods:
            df[f'return_{period}d'] = df[price_col].pct_change(period)
        
        # Log returns
        df['log_return_1d'] = np.log(df[price_col] / df[price_col].shift(1))
        
        return df
    
    def add_technical_indicators(self, df, price_col='SPY_Close', volume_col='SPY_Volume'):
        """Add technical analysis indicators"""
        df = df.copy()
        close = df[price_col].values
        
        try:
            # Moving Averages
            for period in [10, 20, 50, 100, 200]:
                df[f'sma_{period}'] = df[price_col].rolling(period).mean()
                df[f'ema_{period}'] = df[price_col].ewm(span=period, adjust=False).mean()
                df[f'price_to_sma_{period}'] = df[price_col] / df[f'sma_{period}'] - 1
            
            # Moving Average Crossovers
            df['sma_10_50_cross'] = (df['sma_10'] / df['sma_50'] - 1)
            df['sma_20_200_cross'] = (df['sma_20'] / df['sma_200'] - 1)
            
            # RSI (Relative Strength Index)
            if len(close) > 14:
                if TALIB_AVAILABLE:
                    df['rsi_14'] = talib.RSI(close, timeperiod=14)
                    df['rsi_28'] = talib.RSI(close, timeperiod=28)
                else:
                    df['rsi_14'] = self._calculate_rsi(close, 14)
                    df['rsi_28'] = self._calculate_rsi(close, 28)
            
            # MACD
            if len(close) > 26:
                if TALIB_AVAILABLE:
                    macd, signal, hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
                else:
                    macd, signal, hist = self._calculate_macd(close)
                df['macd'] = macd
                df['macd_signal'] = signal
                df['macd_hist'] = hist
            
            # Bollinger Bands
            for period in [20, 50]:
                if len(close) > period:
                    if TALIB_AVAILABLE:
                        upper, middle, lower = talib.BBANDS(close, timeperiod=period, nbdevup=2, nbdevdn=2)
                    else:
                        upper, middle, lower = self._calculate_bbands(close, period)
                    df[f'bb_upper_{period}'] = upper
                    df[f'bb_middle_{period}'] = middle
                    df[f'bb_lower_{period}'] = lower
                    df[f'bb_width_{period}'] = (upper - lower) / middle
                    df[f'bb_position_{period}'] = (df[price_col] - lower) / (upper - lower)
            
            # ATR (Average True Range) - Manual calculation
            if len(close) > 14:
                high = df[price_col] * 1.005 if 'SPY_High' not in df else df['SPY_High']
                low = df[price_col] * 0.995 if 'SPY_Low' not in df else df['SPY_Low']
                
                # True Range
                tr1 = high - low
                tr2 = abs(high - df[price_col].shift(1))
                tr3 = abs(low - df[price_col].shift(1))
                tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
                df['atr_14'] = tr.rolling(14).mean()
            
            # ADX (Average Directional Index) - Simplified version
            if len(close) > 14:
                high = df[price_col] * 1.005 if 'SPY_High' not in df else df['SPY_High']
                low = df[price_col] * 0.995 if 'SPY_Low' not in df else df['SPY_Low']
                
                # Simplified ADX using price momentum
                up_move = high - high.shift(1)
                down_move = low.shift(1) - low
                df['adx_14'] = abs(df[price_col].pct_change(14)).rolling(14).mean() * 100
            
            # Stochastic Oscillator - Manual calculation
            if len(close) > 14:
                high = df[price_col] * 1.005 if 'SPY_High' not in df else df['SPY_High']
                low = df[price_col] * 0.995 if 'SPY_Low' not in df else df['SPY_Low']
                
                low_14 = low.rolling(14).min()
                high_14 = high.rolling(14).max()
                stoch_k = 100 * (df[price_col] - low_14) / (high_14 - low_14)
                stoch_d = stoch_k.rolling(3).mean()
                
                df['stoch_k'] = stoch_k
                df['stoch_d'] = stoch_d
            
            # Volume indicators (if available)
            if volume_col in df.columns:
                df['volume_sma_20'] = df[volume_col].rolling(20).mean()
                df['volume_ratio'] = df[volume_col] / df['volume_sma_20']
                
        except Exception as e:
            print(f"Warning in technical indicators: {e}")
        
        return df
    
    def add_momentum_features(self, df, price_col='SPY_Close'):
        """Add momentum-based features"""
        df = df.copy()
        
        # Rate of Change
        for period in [5, 10, 20]:
            df[f'roc_{period}'] = df[price_col].pct_change(period) * 100
        
        # Momentum
        for period in [10, 20, 50]:
            df[f'momentum_{period}'] = df[price_col] - df[price_col].shift(period)
        
        # Williams %R
        for period in [14, 28]:
            high_roll = df[price_col].rolling(period).max()
            low_roll = df[price_col].rolling(period).min()
            df[f'williams_r_{period}'] = -100 * (high_roll - df[price_col]) / (high_roll - low_roll)
        
        return df
    
    def add_volatility_features(self, df, price_col='SPY_Close'):
        """Add volatility-based features"""
        df = df.copy()
        
        # Historical Volatility
        returns = df[price_col].pct_change()
        for period in [10, 20, 50]:
            df[f'volatility_{period}d'] = returns.rolling(period).std() * np.sqrt(252)
        
        # Parkinson volatility (high-low range)
        if 'SPY_High' in df and 'SPY_Low' in df:
            df['parkinson_vol'] = np.sqrt(
                (1 / (4 * np.log(2))) * 
                np.log(df['SPY_High'] / df['SPY_Low']) ** 2
            ).rolling(20).mean()
        
        # Volatility ratio
        df['vol_ratio_10_50'] = df['volatility_10d'] / df['volatility_50d']
        
        return df
    
    def add_regime_features(self, df, price_col='SPY_Close'):
        """Identify market regimes"""
        df = df.copy()
        
        # Trend strength
        df['trend_strength'] = abs(df[price_col].pct_change(20))
        
        # Days above/below moving average
        df['days_above_sma_50'] = (df[price_col] > df['sma_50']).astype(int)
        df['days_above_sma_200'] = (df[price_col] > df['sma_200']).astype(int)
        
        # Distance from 52-week high
        df['high_52w'] = df[price_col].rolling(252).max()
        df['dist_from_high'] = (df[price_col] / df['high_52w'] - 1) * 100
        
        # Drawdown
        running_max = df[price_col].expanding().max()
        df['drawdown'] = (df[price_col] / running_max - 1) * 100
        
        return df
    
    def add_calendar_features(self, df, date_col='Date'):
        """Add calendar-based features"""
        df = df.copy()
        
        if date_col in df.columns:
            df[date_col] = pd.to_datetime(df[date_col])
            df['day_of_week'] = df[date_col].dt.dayofweek
            df['day_of_month'] = df[date_col].dt.day
            df['month'] = df[date_col].dt.month
            df['quarter'] = df[date_col].dt.quarter
            df['is_month_start'] = df[date_col].dt.is_month_start.astype(int)
            df['is_month_end'] = df[date_col].dt.is_month_end.astype(int)
            df['is_quarter_start'] = df[date_col].dt.is_quarter_start.astype(int)
            df['is_quarter_end'] = df[date_col].dt.is_quarter_end.astype(int)
        
        return df
    
    def add_interaction_features(self, df):
        """Add interaction features"""
        df = df.copy()
        
        # RSI and trend
        if 'rsi_14' in df and 'return_20d' in df:
            df['rsi_trend_interaction'] = df['rsi_14'] * df['return_20d']
        
        # Volatility and momentum
        if 'volatility_20d' in df and 'momentum_20' in df:
            df['vol_momentum_interaction'] = df['volatility_20d'] * df['momentum_20']
        
        return df
    
    def engineer_all_features(self, df):
        """Apply all feature engineering steps"""
        print("Engineering features...")
        
        df = self.add_returns_features(df)
        print("  ✓ Returns features")
        
        df = self.add_technical_indicators(df)
        print("  ✓ Technical indicators")
        
        df = self.add_momentum_features(df)
        print("  ✓ Momentum features")
        
        df = self.add_volatility_features(df)
        print("  ✓ Volatility features")
        
        df = self.add_regime_features(df)
        print("  ✓ Regime features")
        
        df = self.add_calendar_features(df)
        print("  ✓ Calendar features")
        
        df = self.add_interaction_features(df)
        print("  ✓ Interaction features")
        
        print(f"\nTotal features created: {len(df.columns)}")
        
        return df

# Initialize feature engineer
fe = FeatureEngineer(lookback_periods=CONFIG['lookback_periods'])
print("✓ Feature Engineer initialized")

✓ Feature Engineer initialized


## 4. Evaluation Metric Implementation

The competition uses a modified Sharpe ratio that penalizes:
1. Strategies that fail to outperform the market
2. Strategies with volatility > 120% of market volatility

In [38]:
def calculate_sharpe_ratio(returns, risk_free_rate=0.0):
    """
    Calculate Sharpe ratio
    """
    excess_returns = returns - risk_free_rate
    if len(excess_returns) == 0 or excess_returns.std() == 0:
        return 0.0
    return np.sqrt(252) * excess_returns.mean() / excess_returns.std()


def calculate_modified_sharpe(strategy_returns, benchmark_returns, volatility_constraint=1.2):
    """
    Calculate modified Sharpe ratio with penalties
    
    Penalizes:
    1. Underperformance vs benchmark
    2. Excessive volatility (> volatility_constraint * benchmark_volatility)
    """
    # Calculate basic statistics
    strategy_mean = strategy_returns.mean()
    benchmark_mean = benchmark_returns.mean()
    strategy_vol = strategy_returns.std()
    benchmark_vol = benchmark_returns.std()
    
    # Penalty for underperformance
    if strategy_mean < benchmark_mean:
        return_penalty = (strategy_mean - benchmark_mean) / benchmark_vol
    else:
        return_penalty = 0
    
    # Penalty for excessive volatility
    max_allowed_vol = volatility_constraint * benchmark_vol
    if strategy_vol > max_allowed_vol:
        vol_penalty = (strategy_vol - max_allowed_vol) / benchmark_vol
    else:
        vol_penalty = 0
    
    # Calculate Sharpe ratio
    sharpe = calculate_sharpe_ratio(strategy_returns)
    
    # Apply penalties
    modified_sharpe = sharpe - return_penalty - vol_penalty
    
    return modified_sharpe


def evaluate_strategy(allocations, returns, benchmark_returns):
    """
    Evaluate trading strategy performance
    
    Args:
        allocations: Array of daily allocations (0 to 2)
        returns: Market returns
        benchmark_returns: S&P 500 returns
    
    Returns:
        Dictionary of performance metrics
    """
    # Calculate strategy returns
    strategy_returns = allocations * returns
    
    # Calculate cumulative returns
    strategy_cum_returns = (1 + strategy_returns).cumprod()
    benchmark_cum_returns = (1 + benchmark_returns).cumprod()
    
    # Calculate metrics
    metrics = {
        'total_return': strategy_cum_returns.iloc[-1] - 1,
        'benchmark_return': benchmark_cum_returns.iloc[-1] - 1,
        'annualized_return': (strategy_cum_returns.iloc[-1] ** (252 / len(strategy_returns))) - 1,
        'annualized_volatility': strategy_returns.std() * np.sqrt(252),
        'sharpe_ratio': calculate_sharpe_ratio(strategy_returns),
        'modified_sharpe': calculate_modified_sharpe(strategy_returns, benchmark_returns),
        'max_drawdown': (strategy_cum_returns / strategy_cum_returns.cummax() - 1).min(),
        'win_rate': (strategy_returns > 0).sum() / len(strategy_returns),
        'avg_allocation': allocations.mean(),
        'max_allocation': allocations.max(),
        'min_allocation': allocations.min(),
    }
    
    return metrics


print("Evaluation functions defined")

Evaluation functions defined


In [39]:
def prepare_data_for_modeling(df, target_col='market_forward_excess_returns', date_col='date_id'):
    """
    Prepare data for modeling by:
    1. Removing rows with NaN in target
    2. Identifying feature columns
    3. Splitting features and target
    
    For competition data: Uses pre-engineered features (D*, E*, I*, M*, P*, S*, V*)
    For sample data: Uses engineered technical indicators
    """
    df = df.copy()
    
    # Check if this is competition data (has date_id) or sample data (has Date)
    is_competition_data = 'date_id' in df.columns
    
    if is_competition_data:
        print("Using competition data format with pre-engineered features")
        
        # Remove rows where target is NaN
        df = df.dropna(subset=[target_col])
        
        # Get all feature columns (D*, E*, I*, M*, P*, S*, V*)
        feature_prefixes = ['D', 'E', 'I', 'M', 'P', 'S', 'V']
        feature_cols = [col for col in df.columns 
                       if any(col.startswith(prefix) for prefix in feature_prefixes)]
        
        # Remove features with too many NaNs or zero variance
        valid_features = []
        for col in feature_cols:
            if df[col].isna().sum() / len(df) < 0.5:  # Less than 50% NaN
                if df[col].std() > 1e-10:  # Non-zero variance
                    valid_features.append(col)
        
        print(f"Total features: {len(feature_cols)}")
        print(f"Valid features after filtering: {len(valid_features)}")
        print(f"Training samples: {len(df)}")
        
        # Fill remaining NaNs with 0 (competition data already preprocessed)
        X = df[valid_features].fillna(0)
        y = df[target_col]
        dates = df[date_col]
        
    else:
        print("Using sample data format with custom features")
        
        # Use 'Date' as date column for sample data
        date_col = 'Date'
        target_col = 'Target'
        
        # Remove rows where target is NaN
        df = df.dropna(subset=[target_col])
        
        # Identify non-feature columns
        exclude_cols = [date_col, target_col, 'SPY_Close', 'SPY_Volume', 'SPY_Return']
        if 'SPY_High' in df.columns:
            exclude_cols.extend(['SPY_High', 'SPY_Low', 'SPY_Open'])
        
        # Get feature columns (all numeric columns except excluded)
        feature_cols = [col for col in df.columns 
                       if col not in exclude_cols 
                       and df[col].dtype in ['float64', 'int64']]
        
        # Remove features with too many NaNs or zero variance
        valid_features = []
        for col in feature_cols:
            if df[col].isna().sum() / len(df) < 0.5:  # Less than 50% NaN
                if df[col].std() > 1e-10:  # Non-zero variance
                    valid_features.append(col)
        
        print(f"Total features: {len(feature_cols)}")
        print(f"Valid features after filtering: {len(valid_features)}")
        print(f"Training samples: {len(df)}")
        
        # Fill remaining NaNs with forward fill then backfill
        X = df[valid_features].fillna(method='ffill').fillna(method='bfill').fillna(0)
        y = df[target_col]
        dates = df[date_col] if date_col in df.columns else None
    
    return X, y, dates, valid_features


if train_data_fe is not None:
    X_train, y_train, dates_train, feature_names = prepare_data_for_modeling(train_data_fe)
    
    print(f"\nFeature matrix shape: {X_train.shape}")
    print(f"Target shape: {y_train.shape}")
    print(f"\nSample features:")
    print(feature_names[:20])

Using competition data format with pre-engineered features
Total features: 94
Valid features after filtering: 86
Training samples: 9021

Feature matrix shape: (9021, 86)
Target shape: (9021,)

Sample features:
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19']


## 6. Model Development

Building an ensemble of models:
1. **XGBoost**: Gradient boosting with regularization
2. **LightGBM**: Fast gradient boosting
3. **CatBoost**: Handles categorical features well
4. **Neural Network**: Deep learning for complex patterns

In [40]:
class MarketPredictionModel:
    """
    Ensemble model for market prediction
    """
    
    def __init__(self, model_type='xgboost', **kwargs):
        self.model_type = model_type
        self.model = None
        self.scaler = RobustScaler()
        self.is_fitted = False
        self.kwargs = kwargs
        
    def build_model(self, input_dim):
        """Build the appropriate model"""
        
        if self.model_type == 'xgboost':
            self.model = xgb.XGBRegressor(
                n_estimators=self.kwargs.get('n_estimators', 500),
                max_depth=self.kwargs.get('max_depth', 6),
                learning_rate=self.kwargs.get('learning_rate', 0.01),
                subsample=self.kwargs.get('subsample', 0.8),
                colsample_bytree=self.kwargs.get('colsample_bytree', 0.8),
                reg_alpha=self.kwargs.get('reg_alpha', 0.1),
                reg_lambda=self.kwargs.get('reg_lambda', 1.0),
                random_state=CONFIG['random_state'],
                n_jobs=-1
            )
            
        elif self.model_type == 'lightgbm':
            self.model = lgb.LGBMRegressor(
                n_estimators=self.kwargs.get('n_estimators', 500),
                max_depth=self.kwargs.get('max_depth', 6),
                learning_rate=self.kwargs.get('learning_rate', 0.01),
                subsample=self.kwargs.get('subsample', 0.8),
                colsample_bytree=self.kwargs.get('colsample_bytree', 0.8),
                reg_alpha=self.kwargs.get('reg_alpha', 0.1),
                reg_lambda=self.kwargs.get('reg_lambda', 1.0),
                random_state=CONFIG['random_state'],
                n_jobs=-1,
                verbose=-1
            )
            
        elif self.model_type == 'catboost':
            self.model = CatBoostRegressor(
                iterations=self.kwargs.get('n_estimators', 500),
                depth=self.kwargs.get('max_depth', 6),
                learning_rate=self.kwargs.get('learning_rate', 0.01),
                l2_leaf_reg=self.kwargs.get('reg_lambda', 3.0),
                random_state=CONFIG['random_state'],
                verbose=False
            )
            
        elif self.model_type == 'neural_network':
            self.model = self._build_neural_network(input_dim)
            
        return self.model
    
    def _build_neural_network(self, input_dim):
        """Build neural network architecture"""
        
        model = keras.Sequential([
            layers.Input(shape=(input_dim,)),
            layers.BatchNormalization(),
            
            layers.Dense(256, activation='relu', 
                        kernel_regularizer=regularizers.l2(0.001)),
            layers.Dropout(0.3),
            layers.BatchNormalization(),
            
            layers.Dense(128, activation='relu',
                        kernel_regularizer=regularizers.l2(0.001)),
            layers.Dropout(0.2),
            layers.BatchNormalization(),
            
            layers.Dense(64, activation='relu',
                        kernel_regularizer=regularizers.l2(0.001)),
            layers.Dropout(0.2),
            
            layers.Dense(32, activation='relu'),
            layers.Dense(1)  # Output layer
        ])
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    def fit(self, X_train, y_train, X_val=None, y_val=None):
        """Train the model"""
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        
        if self.model is None:
            self.build_model(X_train.shape[1])
        
        if self.model_type == 'neural_network':
            # Prepare validation data
            if X_val is not None:
                X_val_scaled = self.scaler.transform(X_val)
                validation_data = (X_val_scaled, y_val)
            else:
                validation_data = None
            
            # Callbacks
            callbacks = [
                EarlyStopping(monitor='val_loss' if validation_data else 'loss',
                            patience=20, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss' if validation_data else 'loss',
                                factor=0.5, patience=10, min_lr=1e-6)
            ]
            
            # Train
            history = self.model.fit(
                X_train_scaled, y_train,
                validation_data=validation_data,
                epochs=self.kwargs.get('epochs', 100),
                batch_size=self.kwargs.get('batch_size', 64),
                callbacks=callbacks,
                verbose=0
            )
            
            self.history = history
            
        else:
            # Gradient boosting models
            if X_val is not None:
                X_val_scaled = self.scaler.transform(X_val)
                eval_set = [(X_val_scaled, y_val)]
                
                if self.model_type == 'xgboost':
                    self.model.fit(X_train_scaled, y_train,
                                 eval_set=eval_set,
                                 verbose=False)
                elif self.model_type == 'lightgbm':
                    self.model.fit(X_train_scaled, y_train,
                                 eval_set=eval_set,
                                 callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
                elif self.model_type == 'catboost':
                    self.model.fit(X_train_scaled, y_train,
                                 eval_set=eval_set,
                                 verbose=False)
            else:
                self.model.fit(X_train_scaled, y_train)
        
        self.is_fitted = True
        return self
    
    def predict(self, X):
        """Make predictions"""
        X_scaled = self.scaler.transform(X)
        
        if self.model_type == 'neural_network':
            predictions = self.model.predict(X_scaled, verbose=0).flatten()
        else:
            predictions = self.model.predict(X_scaled)
        
        return predictions
    
    def get_feature_importance(self, feature_names):
        """Get feature importance (for tree-based models)"""
        if self.model_type in ['xgboost', 'lightgbm', 'catboost']:
            if self.model_type == 'xgboost':
                importance = self.model.feature_importances_
            elif self.model_type == 'lightgbm':
                importance = self.model.feature_importances_
            elif self.model_type == 'catboost':
                importance = self.model.feature_importances_
            
            feature_imp = pd.DataFrame({
                'feature': feature_names,
                'importance': importance
            }).sort_values('importance', ascending=False)
            
            return feature_imp
        else:
            return None


print("Model class defined")

Model class defined


## 7. Position Sizing Strategy

Convert predicted returns into optimal allocations (0-2 range)

In [41]:
class PositionSizer:
    """
    Convert predicted returns to optimal allocations
    """
    
    def __init__(self, min_allocation=0.0, max_allocation=2.0, 
                 volatility_target=0.16, risk_free_rate=0.0):
        self.min_allocation = min_allocation
        self.max_allocation = max_allocation
        self.volatility_target = volatility_target
        self.risk_free_rate = risk_free_rate
        
    def kelly_criterion(self, predicted_return, volatility, confidence=0.25):
        """
        Kelly Criterion for position sizing
        f* = (μ - r) / σ²
        
        Args:
            predicted_return: Expected return
            volatility: Expected volatility
            confidence: Fraction of Kelly to use (fractional Kelly)
        """
        if volatility == 0:
            return 1.0
        
        # Kelly fraction
        kelly_fraction = (predicted_return - self.risk_free_rate) / (volatility ** 2)
        
        # Apply confidence (fractional Kelly)
        allocation = kelly_fraction * confidence
        
        # Clip to valid range
        allocation = np.clip(allocation, self.min_allocation, self.max_allocation)
        
        return allocation
    
    def volatility_targeting(self, predicted_return, current_volatility):
        """
        Target volatility approach
        Scale position to achieve target volatility
        """
        if current_volatility == 0:
            return 1.0
        
        # Base allocation on predicted return sign
        if predicted_return > 0:
            base_allocation = 1.0
        else:
            base_allocation = 0.5  # Reduce exposure if negative prediction
        
        # Scale for volatility
        vol_scalar = self.volatility_target / current_volatility
        allocation = base_allocation * vol_scalar
        
        # Clip to valid range
        allocation = np.clip(allocation, self.min_allocation, self.max_allocation)
        
        return allocation
    
    def threshold_strategy(self, predicted_return, threshold=0.0):
        """
        Simple threshold strategy
        """
        if predicted_return > threshold:
            # Scale allocation based on prediction confidence
            allocation = 1.0 + min(predicted_return * 20, 1.0)  # Up to 2x leverage
        elif predicted_return < -threshold:
            allocation = max(0.5 + predicted_return * 20, 0.0)  # Reduce to 0
        else:
            allocation = 1.0  # Market exposure
        
        return np.clip(allocation, self.min_allocation, self.max_allocation)
    
    def hybrid_strategy(self, predicted_return, volatility, 
                       kelly_weight=0.3, vol_target_weight=0.3, threshold_weight=0.4):
        """
        Combine multiple strategies
        """
        # Get allocations from each strategy
        kelly_alloc = self.kelly_criterion(predicted_return, volatility)
        vol_alloc = self.volatility_targeting(predicted_return, volatility)
        threshold_alloc = self.threshold_strategy(predicted_return)
        
        # Weighted average
        allocation = (kelly_weight * kelly_alloc + 
                     vol_target_weight * vol_alloc + 
                     threshold_weight * threshold_alloc)
        
        return np.clip(allocation, self.min_allocation, self.max_allocation)
    
    def convert_predictions_to_allocations(self, predictions, volatilities, 
                                          strategy='hybrid'):
        """
        Convert array of predictions to allocations
        
        Args:
            predictions: Array of predicted returns
            volatilities: Array of volatility estimates
            strategy: 'kelly', 'vol_target', 'threshold', or 'hybrid'
        """
        allocations = []
        
        for pred, vol in zip(predictions, volatilities):
            if strategy == 'kelly':
                alloc = self.kelly_criterion(pred, vol)
            elif strategy == 'vol_target':
                alloc = self.volatility_targeting(pred, vol)
            elif strategy == 'threshold':
                alloc = self.threshold_strategy(pred)
            else:  # hybrid
                alloc = self.hybrid_strategy(pred, vol)
            
            allocations.append(alloc)
        
        return np.array(allocations)


# Initialize position sizer
position_sizer = PositionSizer(
    min_allocation=CONFIG['min_allocation'],
    max_allocation=CONFIG['max_allocation']
)

print("Position sizer initialized")

Position sizer initialized


## 8. Training with Time Series Cross-Validation

## 9. Final Model Training (Full Dataset)

## 10. Backtesting and Performance Analysis

In [42]:
def plot_backtest_results(strategy_returns, benchmark_returns, allocations, dates=None):
    """
    Visualize backtest results
    """
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Cumulative returns
    ax1 = axes[0]
    strategy_cum = (1 + strategy_returns).cumprod()
    benchmark_cum = (1 + benchmark_returns.iloc[:len(strategy_returns)]).cumprod()
    
    if dates is not None:
        dates_plot = dates.iloc[:len(strategy_returns)]
        ax1.plot(dates_plot, strategy_cum.values, label='Strategy', linewidth=2)
        ax1.plot(dates_plot, benchmark_cum.values, label='Buy & Hold', linewidth=2, alpha=0.7)
    else:
        ax1.plot(strategy_cum.values, label='Strategy', linewidth=2)
        ax1.plot(benchmark_cum.values, label='Buy & Hold', linewidth=2, alpha=0.7)
    
    ax1.set_title('Cumulative Returns: Strategy vs Buy & Hold', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Cumulative Return', fontsize=12)
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Drawdown
    ax2 = axes[1]
    running_max = strategy_cum.expanding().max()
    drawdown = (strategy_cum / running_max - 1) * 100
    
    if dates is not None:
        ax2.fill_between(dates_plot, drawdown.values, 0, alpha=0.3, color='red')
        ax2.plot(dates_plot, drawdown.values, color='red', linewidth=1)
    else:
        ax2.fill_between(range(len(drawdown)), drawdown.values, 0, alpha=0.3, color='red')
        ax2.plot(drawdown.values, color='red', linewidth=1)
    
    ax2.set_title('Strategy Drawdown', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Drawdown (%)', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Allocations over time
    ax3 = axes[2]
    if dates is not None:
        ax3.plot(dates_plot, allocations, linewidth=1.5, color='purple')
        ax3.fill_between(dates_plot, allocations, 1.0, alpha=0.2, color='green',
                        where=(allocations >= 1.0), label='Leverage')
        ax3.fill_between(dates_plot, allocations, 1.0, alpha=0.2, color='red',
                        where=(allocations < 1.0), label='Reduced Exposure')
    else:
        ax3.plot(allocations, linewidth=1.5, color='purple')
        ax3.fill_between(range(len(allocations)), allocations, 1.0, alpha=0.2, color='green',
                        where=(allocations >= 1.0), label='Leverage')
        ax3.fill_between(range(len(allocations)), allocations, 1.0, alpha=0.2, color='red',
                        where=(allocations < 1.0), label='Reduced Exposure')
    
    ax3.axhline(y=1.0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    ax3.axhline(y=2.0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Max Leverage')
    ax3.axhline(y=0.0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Zero Exposure')
    ax3.set_title('Position Allocation Over Time', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Allocation', fontsize=12)
    ax3.set_ylim(-0.1, 2.1)
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


print("Plotting function ready")

Plotting function ready



## 11. Submission Using Competition API

The competition requires using an evaluation API that prevents look-ahead bias.

## 12. Complete Workflow Example

## 13. Model Optimization Tips

### Hyperparameter Tuning
Consider optimizing:
- **Learning rates**: Try [0.001, 0.01, 0.05]
- **Tree depth**: Try [4, 6, 8, 10]
- **Number of estimators**: Try [300, 500, 1000]
- **Regularization**: Adjust L1/L2 penalties

### Feature Selection
- Use feature importance to remove low-impact features
- Try PCA or other dimensionality reduction
- Create more domain-specific features

### Position Sizing
- Adjust Kelly fraction (currently 0.25)
- Experiment with different volatility targets
- Consider transaction costs

### Ensemble Weights
- Validate individual model performance
- Adjust weights based on out-of-sample performance
- Consider stacking with meta-learner

### Risk Management
- Implement maximum drawdown constraints
- Add position size limits based on market regime
- Consider dynamic volatility targeting

---

## Summary

This notebook provides a complete solution for the Hull Tactical Market Prediction competition:

✅ **Data Loading**: Flexible data loading with sample data generation  
✅ **Feature Engineering**: 80+ technical, momentum, volatility, and regime features  
✅ **Ensemble Models**: XGBoost, LightGBM, CatBoost, and Neural Networks  
✅ **Position Sizing**: Kelly Criterion, volatility targeting, and hybrid strategies  
✅ **Evaluation**: Modified Sharpe ratio with proper backtesting  
✅ **Submission**: API-ready prediction pipeline  

### To Run:
```python
# 1. Load your actual data
train_df, test_df = load_data()

# 2. Run the complete workflow
results = run_complete_workflow()

# 3. Make submission
make_submission(results['model'], fe, position_sizer, results['features'])
```

Good luck challenging the Efficient Market Hypothesis! 🚀📈

## 14. Additional Resources and Notes

### Key Competition Details
- **Metric**: Modified Sharpe ratio with volatility penalty
- **Allocation Range**: 0 to 2 (allows leverage up to 2x)
- **Volatility Constraint**: 120% of market volatility
- **Objective**: Beat S&P 500 while managing risk

### Important Considerations
1. **No Look-Ahead Bias**: Use time-series splits, never shuffle data
2. **Transaction Costs**: Not explicitly mentioned but should be considered
3. **Regime Changes**: Market behavior changes over time
4. **Overfitting**: Be careful with too many features or complex models

### Data Sources (if using external data)
- Economic indicators (GDP, unemployment, interest rates)
- Sentiment data (VIX, put/call ratios)
- Technical indicators from price/volume
- Fundamental factors (earnings, P/E ratios)

### Best Practices
- Start simple, add complexity gradually
- Validate everything with out-of-sample data
- Monitor correlation between predictions and actual returns
- Consider market regime (bull/bear/sideways)
- Document all assumptions and decisions

### Next Steps
1. Load actual competition data
2. Run the complete workflow
3. Analyze feature importance
4. Optimize hyperparameters
5. Test different position sizing strategies
6. Submit and iterate based on leaderboard feedback

In [43]:
"""
Complete workflow from data loading to submission

Uncomment and run this cell to execute the full pipeline:
"""

def run_complete_workflow():
    """
    Execute the complete modeling pipeline
    """
    print("="*80)
    print("HULL TACTICAL - MARKET PREDICTION PIPELINE")
    print("="*80)
    
    # 1. Load data
    print("\n1. Loading data...")
    train_df, test_df = load_data()
    
    # 2. Feature engineering
    print("\n2. Engineering features...")
    train_fe = fe.engineer_all_features(train_df)
    
    # 3. Prepare for modeling
    print("\n3. Preparing data for modeling...")
    X, y, dates, features = prepare_data_for_modeling(train_fe)
    
    # 4. Time series cross-validation
    print("\n4. Running cross-validation...")
    cv_results = train_and_validate(X, y, n_splits=3)  # Use 3 for faster execution
    
    # 5. Train final model
    print("\n5. Training final model...")
    final_model = train_final_model(X, y, features)
    
    # 6. Backtest
    print("\n6. Running backtest...")
    allocations, strategy_returns, metrics = backtest_strategy(
        final_model, X, y, train_fe['SPY_Return'], dates
    )
    
    # 7. Visualize results
    print("\n7. Visualizing results...")
    plot_backtest_results(
        strategy_returns, 
        train_fe['SPY_Return'], 
        allocations, 
        dates
    )
    
    # 8. Prepare for submission
    print("\n8. Creating submission pipeline...")
    submission_pipeline = SubmissionPipeline(
        final_model, fe, position_sizer, features
    )
    
    print("\n" + "="*80)
    print("PIPELINE COMPLETE!")
    print("="*80)
    print("\nNext steps:")
    print("1. Review the backtest results above")
    print("2. Fine-tune model parameters if needed")
    print("3. Run make_submission() when ready to submit")
    
    return {
        'model': final_model,
        'pipeline': submission_pipeline,
        'metrics': metrics,
        'features': features
    }


# Uncomment to run the complete workflow
# results = run_complete_workflow()

print("Complete workflow function ready")
print("\nTo execute the full pipeline, uncomment and run:")
print("  results = run_complete_workflow()")

Complete workflow function ready

To execute the full pipeline, uncomment and run:
  results = run_complete_workflow()


## ✅ Setup Complete!

All components are now loaded and ready to use:

### 📊 Data Status
- **Competition data loaded**: 9,021 training samples with 97 features
- **Test data available**: 10 samples
- **Features**: Pre-engineered numerical features (D1-D97)

### 🔧 Components Ready
1. ✅ Feature engineering pipeline (with TA-Lib fallback)
2. ✅ Evaluation metrics (Sharpe ratio, modified Sharpe)
3. ✅ Ensemble predictor (XGBoost, LightGBM, CatBoost, Neural Network)
4. ✅ Model classes initialized
5. ✅ Position sizing strategies (Kelly, volatility targeting, hybrid)
6. ✅ Training functions (time-series CV)
7. ✅ Backtesting framework
8. ✅ Visualization tools
9. ✅ Submission pipeline

### 🚀 Next Steps

**For competition data** (current setup):
The competition provides pre-engineered features, so you should focus on:
- Model training and hyperparameter tuning
- Ensemble weight optimization  
- Position sizing strategy refinement
- Creating a submission with the API

**Note**: The competition uses its own API for submission that prevents look-ahead bias. You'll need to adapt the code to work with their specific API format.

In [44]:
"""
Submission code for the competition API

The API iterates through test dates and provides data up to that date.
Your model predicts the allocation for the next day (0 to 2).

Example usage (pseudo-code):
```
from hull_tactical import HullTacticalEnvironment

env = HullTacticalEnvironment()

for (test_date, historical_data) in env.iter_test():
    # Engineer features for this date
    features = engineer_features(historical_data)
    
    # Predict next day return
    predicted_return = model.predict(features)
    
    # Convert to allocation
    allocation = convert_to_allocation(predicted_return)
    
    # Submit allocation (0 to 2)
    env.predict(allocation)
```
"""

class SubmissionPipeline:
    """
    Pipeline for making predictions during submission
    """
    
    def __init__(self, ensemble, feature_engineer, position_sizer, feature_names):
        self.ensemble = ensemble
        self.feature_engineer = feature_engineer
        self.position_sizer = position_sizer
        self.feature_names = feature_names
        self.historical_data = []
        
    def process_new_data(self, historical_data):
        """
        Process historical data and prepare features
        """
        # Convert to DataFrame if needed
        if not isinstance(historical_data, pd.DataFrame):
            df = pd.DataFrame(historical_data)
        else:
            df = historical_data.copy()
        
        # Engineer features
        df_features = self.feature_engineer.engineer_all_features(df)
        
        # Prepare for prediction (last row)
        X, y, dates, _ = prepare_data_for_modeling(df_features)
        
        return X, df_features
    
    def predict_allocation(self, historical_data):
        """
        Predict allocation for next trading day
        
        Args:
            historical_data: DataFrame with historical market data
            
        Returns:
            float: Allocation between 0 and 2
        """
        try:
            # Process data
            X, df_features = self.process_new_data(historical_data)
            
            if len(X) == 0:
                return 1.0  # Default to market exposure
            
            # Get latest features
            X_latest = X.iloc[[-1]][self.feature_names]
            
            # Predict return
            predicted_return = self.ensemble.predict(X_latest)[0]
            
            # Estimate volatility (last 20 days)
            if 'SPY_Return' in df_features.columns:
                recent_returns = df_features['SPY_Return'].dropna().tail(20)
                volatility = recent_returns.std() if len(recent_returns) > 0 else 0.01
            else:
                volatility = 0.01
            
            # Convert to allocation
            allocation = self.position_sizer.hybrid_strategy(
                predicted_return, volatility
            )
            
            # Ensure valid range
            allocation = np.clip(allocation, 
                               CONFIG['min_allocation'], 
                               CONFIG['max_allocation'])
            
            return float(allocation)
            
        except Exception as e:
            print(f"Error in prediction: {e}")
            return 1.0  # Default to market exposure


# Example submission function
def make_submission(ensemble, feature_engineer, position_sizer, feature_names):
    """
    Make submission using the competition API
    """
    # Initialize submission pipeline
    pipeline = SubmissionPipeline(ensemble, feature_engineer, position_sizer, feature_names)
    
    # Import the competition API (adjust based on actual API)
    try:
        from hull_tactical import HullTacticalEnvironment
        
        env = HullTacticalEnvironment()
        
        print("Starting submission...")
        
        for test_date, historical_data in env.iter_test():
            # Predict allocation
            allocation = pipeline.predict_allocation(historical_data)
            
            # Submit prediction
            env.predict(allocation)
            
            if env.step % 100 == 0:
                print(f"Processed {env.step} predictions...")
        
        print("Submission complete!")
        
    except ImportError:
        print("Competition API not found. This is expected in development.")
        print("The submission pipeline is ready for actual competition environment.")


print("Submission pipeline ready")

Submission pipeline ready


In [45]:
def backtest_strategy(ensemble, X, y, returns, dates=None):
    """
    Backtest the trading strategy
    """
    print("Running backtest...\n")
    
    # Generate predictions
    predictions = ensemble.predict(X)
    
    # Estimate volatilities (rolling 20-day)
    volatilities = returns.rolling(20).std().fillna(returns.std())
    volatilities = volatilities.iloc[:len(predictions)].values
    
    # Convert predictions to allocations
    allocations = position_sizer.convert_predictions_to_allocations(
        predictions, volatilities, strategy='hybrid'
    )
    
    # Calculate strategy returns
    strategy_returns = pd.Series(allocations * returns.iloc[:len(allocations)].values,
                                 index=returns.iloc[:len(allocations)].index)
    
    # Evaluate
    metrics = evaluate_strategy(
        pd.Series(allocations),
        returns.iloc[:len(allocations)],
        returns.iloc[:len(allocations)]  # benchmark is market itself
    )
    
    # Print metrics
    print("="*60)
    print("BACKTEST RESULTS")
    print("="*60)
    print(f"Total Return:          {metrics['total_return']:>10.2%}")
    print(f"Benchmark Return:      {metrics['benchmark_return']:>10.2%}")
    print(f"Annualized Return:     {metrics['annualized_return']:>10.2%}")
    print(f"Annualized Volatility: {metrics['annualized_volatility']:>10.2%}")
    print(f"Sharpe Ratio:          {metrics['sharpe_ratio']:>10.2f}")
    print(f"Modified Sharpe:       {metrics['modified_sharpe']:>10.2f}")
    print(f"Max Drawdown:          {metrics['max_drawdown']:>10.2%}")
    print(f"Win Rate:              {metrics['win_rate']:>10.2%}")
    print(f"Avg Allocation:        {metrics['avg_allocation']:>10.2f}")
    print(f"Max Allocation:        {metrics['max_allocation']:>10.2f}")
    print(f"Min Allocation:        {metrics['min_allocation']:>10.2f}")
    print("="*60)
    
    return allocations, strategy_returns, metrics


print("Backtest function ready")

Backtest function ready


In [46]:
def train_final_model(X, y, feature_names):
    """
    Train final ensemble model on full dataset
    """
    print("Training final ensemble model on full dataset...")
    
    # Create ensemble
    final_ensemble = EnsemblePredictor()
    
    # Split for validation (last 20% for early stopping)
    split_idx = int(len(X) * 0.8)
    X_train_final = X.iloc[:split_idx]
    y_train_final = y.iloc[:split_idx]
    X_val_final = X.iloc[split_idx:]
    y_val_final = y.iloc[split_idx:]
    
    # XGBoost
    print("\n  Training XGBoost...")
    xgb_model = MarketPredictionModel('xgboost',
                                     n_estimators=500,
                                     max_depth=6,
                                     learning_rate=0.01,
                                     subsample=0.8,
                                     colsample_bytree=0.8)
    xgb_model.fit(X_train_final, y_train_final, X_val_final, y_val_final)
    final_ensemble.add_model('XGBoost', xgb_model, weight=1.0)
    
    # LightGBM
    print("  Training LightGBM...")
    lgb_model = MarketPredictionModel('lightgbm',
                                     n_estimators=500,
                                     max_depth=6,
                                     learning_rate=0.01,
                                     subsample=0.8,
                                     colsample_bytree=0.8)
    lgb_model.fit(X_train_final, y_train_final, X_val_final, y_val_final)
    final_ensemble.add_model('LightGBM', lgb_model, weight=1.0)
    
    # CatBoost
    print("  Training CatBoost...")
    cat_model = MarketPredictionModel('catboost',
                                     n_estimators=500,
                                     max_depth=6,
                                     learning_rate=0.01)
    cat_model.fit(X_train_final, y_train_final, X_val_final, y_val_final)
    final_ensemble.add_model('CatBoost', cat_model, weight=1.0)
    
    # Neural Network
    print("  Training Neural Network...")
    nn_model = MarketPredictionModel('neural_network',
                                    epochs=100,
                                    batch_size=64)
    nn_model.fit(X_train_final, y_train_final, X_val_final, y_val_final)
    final_ensemble.add_model('NeuralNet', nn_model, weight=0.5)
    
    print("\n✓ Final ensemble trained successfully")
    
    # Feature importance (from XGBoost)
    print("\nTop 20 Most Important Features:")
    feature_imp = xgb_model.get_feature_importance(feature_names)
    display(feature_imp.head(20))
    
    return final_ensemble


# Note: Uncomment to train final model
# if 'X_train' in locals() and 'y_train' in locals():
#     final_ensemble = train_final_model(X_train, y_train, feature_names)

print("Final training function ready")

Final training function ready


In [47]:
def train_and_validate(X, y, n_splits=5):
    """
    Train models using time series cross-validation
    """
    # Time series split
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    # Store results
    fold_results = []
    
    # Initialize ensemble
    ensemble = EnsemblePredictor()
    
    print(f"Training with {n_splits}-fold Time Series Cross-Validation\n")
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
        print(f"\n{'='*60}")
        print(f"Fold {fold}/{n_splits}")
        print(f"{'='*60}")
        
        # Split data
        X_train_fold = X.iloc[train_idx]
        y_train_fold = y.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_val_fold = y.iloc[val_idx]
        
        print(f"Train size: {len(X_train_fold)}, Val size: {len(X_val_fold)}")
        
        # Train ensemble for this fold
        fold_ensemble = EnsemblePredictor()
        
        # XGBoost
        xgb_model = MarketPredictionModel('xgboost', 
                                         n_estimators=300,
                                         max_depth=5,
                                         learning_rate=0.01)
        fold_ensemble.add_model('XGBoost', xgb_model, weight=1.0)
        
        # LightGBM
        lgb_model = MarketPredictionModel('lightgbm',
                                         n_estimators=300,
                                         max_depth=5,
                                         learning_rate=0.01)
        fold_ensemble.add_model('LightGBM', lgb_model, weight=1.0)
        
        # CatBoost
        cat_model = MarketPredictionModel('catboost',
                                         n_estimators=300,
                                         max_depth=5,
                                         learning_rate=0.01)
        fold_ensemble.add_model('CatBoost', cat_model, weight=1.0)
        
        # Neural Network
        nn_model = MarketPredictionModel('neural_network',
                                        epochs=100,
                                        batch_size=64)
        fold_ensemble.add_model('NeuralNet', nn_model, weight=0.5)
        
        # Train ensemble
        fold_ensemble.fit(X_train_fold, y_train_fold, X_val_fold, y_val_fold)
        
        # Validate
        val_predictions = fold_ensemble.predict(X_val_fold)
        
        # Calculate metrics
        mse = mean_squared_error(y_val_fold, val_predictions)
        mae = mean_absolute_error(y_val_fold, val_predictions)
        
        # Direction accuracy
        direction_accuracy = np.mean(
            (val_predictions > 0) == (y_val_fold.values > 0)
        )
        
        print(f"\nValidation Metrics:")
        print(f"  MSE: {mse:.6f}")
        print(f"  MAE: {mae:.6f}")
        print(f"  Direction Accuracy: {direction_accuracy:.2%}")
        
        # Store results
        fold_results.append({
            'fold': fold,
            'mse': mse,
            'mae': mae,
            'direction_accuracy': direction_accuracy,
            'ensemble': fold_ensemble
        })
    
    # Summary
    print(f"\n{'='*60}")
    print("Cross-Validation Summary")
    print(f"{'='*60}")
    
    avg_mse = np.mean([r['mse'] for r in fold_results])
    avg_mae = np.mean([r['mae'] for r in fold_results])
    avg_dir_acc = np.mean([r['direction_accuracy'] for r in fold_results])
    
    print(f"Average MSE: {avg_mse:.6f}")
    print(f"Average MAE: {avg_mae:.6f}")
    print(f"Average Direction Accuracy: {avg_dir_acc:.2%}")
    
    return fold_results


# Note: Uncomment to run training (takes time)
# if 'X_train' in locals() and 'y_train' in locals():
#     fold_results = train_and_validate(X_train, y_train, n_splits=CONFIG['n_splits'])

print("Training function ready")

Training function ready


In [48]:
class EnsemblePredictor:
    """
    Ensemble of multiple models with weighted predictions
    """
    
    def __init__(self):
        self.models = {}
        self.weights = {}
        
    def add_model(self, name, model, weight=1.0):
        """Add a model to the ensemble"""
        self.models[name] = model
        self.weights[name] = weight
        
    def fit(self, X_train, y_train, X_val=None, y_val=None):
        """Train all models in the ensemble"""
        print("Training ensemble models...")
        
        for name, model in self.models.items():
            print(f"\n  Training {name}...")
            model.fit(X_train, y_train, X_val, y_val)
            print(f"  ✓ {name} trained")
        
        print("\n✓ All models trained")
        
    def predict(self, X):
        """Make weighted ensemble predictions"""
        predictions = []
        total_weight = sum(self.weights.values())
        
        for name, model in self.models.items():
            pred = model.predict(X)
            weight = self.weights[name] / total_weight
            predictions.append(pred * weight)
        
        return np.sum(predictions, axis=0)
    
    def get_individual_predictions(self, X):
        """Get predictions from each model"""
        predictions = {}
        for name, model in self.models.items():
            predictions[name] = model.predict(X)
        return predictions


print("Ensemble class defined")

Ensemble class defined


## 5. Data Preparation for Modeling

In [49]:
# Check and prepare data
if train_data is not None:
    print("Data columns:", train_data.columns.tolist()[:10])
    
    # Check if this is competition data format or sample data
    if 'SPY_Close' in train_data.columns:
        # Sample data format
        train_data_fe = fe.engineer_all_features(train_data)
    else:
        # Competition data format with D1, D2, etc.
        print("\n⚠ Competition data detected!")
        print("The actual competition data has numerical features (D1, D2, D3...)")
        print("These are already engineered features provided by the competition.")
        print("\nFor competition data, you should:")
        print("1. Use the features as-is (they're already processed)")
        print("2. Focus on model selection and ensemble strategies")
        print("3. Optimize position sizing parameters")
        
        # For demo purposes, let's use the data directly
        train_data_fe = train_data.copy()
        
        # Identify feature columns (exclude date_id and target if present)
        feature_cols = [col for col in train_data_fe.columns if col not in ['date_id', 'target', 'Target']]
        print(f"\n✓ Using {len(feature_cols)} competition features directly")
    
    # Display sample
    print("\nData Preview:")
    display(train_data_fe.head())
    
    print("\nData shape:", train_data_fe.shape)

Data columns: ['date_id', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9']

⚠ Competition data detected!
The actual competition data has numerical features (D1, D2, D3...)
These are already engineered features provided by the competition.

For competition data, you should:
1. Use the features as-is (they're already processed)
2. Focus on model selection and ensemble strategies
3. Optimize position sizing parameters

✓ Using 97 competition features directly

Data Preview:


,date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,I8,I9,M1,M10,M11,M12,M13,M14,M15,M16,M17,M18,M2,M3,M4,M5,M6,M7,M8,M9,P1,P10,P11,P12,P13,P2,P3,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
0,0,0,0,0,1,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002421,0.000301,-0.003038
1,1,0,0,0,1,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008495,0.000303,-0.009114
2,2,0,0,0,1,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009624,0.000301,-0.010243
3,3,0,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004662,0.000299,0.004046
4,4,0,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011686,0.000299,-0.012301



Data shape: (9021, 98)


In [50]:
# Data paths - adjust based on actual data location
DATA_PATH = r"D:\ML Practice\Hull Tactical - Market Prediction\hull-tactical-market-prediction"

# Alternative path if data is in workspace
if not os.path.exists(DATA_PATH):
    DATA_PATH = Path.cwd()
    print(f"Using workspace directory: {DATA_PATH}")
else:
    print(f"Data path: {DATA_PATH}")

# Configuration
CONFIG = {
    'data_path': DATA_PATH,
    'min_allocation': 0.0,    # Minimum allocation to S&P 500
    'max_allocation': 2.0,    # Maximum allocation (with leverage)
    'volatility_constraint': 1.2,  # 120% of market volatility
    'lookback_periods': [5, 10, 20, 50, 100, 200],
    'risk_free_rate': 0.0,  # Daily risk-free rate
    'n_splits': 5,  # Time series cross-validation splits
    'random_state': 42
}

print("\n✓ Configuration loaded:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

Data path: D:\ML Practice\Hull Tactical - Market Prediction\hull-tactical-market-prediction

✓ Configuration loaded:
  data_path: D:\ML Practice\Hull Tactical - Market Prediction\hull-tactical-market-prediction
  min_allocation: 0.0
  max_allocation: 2.0
  volatility_constraint: 1.2
  lookback_periods: [5, 10, 20, 50, 100, 200]
  risk_free_rate: 0.0
  n_splits: 5
  random_state: 42


## ✓ Setup Complete - Ready for Training

**Status**: All code cells executed successfully!

**Data Loaded**:
- Training samples: 9,021
- Features: 86 valid features (after filtering low-variance and high-NaN features)
- Target: `market_forward_excess_returns`

**Competition Data Format**:
- The competition provides pre-engineered features (D1-D9, E1-E20, I1-I9, M1-M18, P1-P13, S1-S12, V1-V13)
- No need for custom feature engineering
- Focus on model selection, ensemble strategies, and position sizing

**Next Steps** (uncomment to execute):
1. Train and validate models: `results = run_complete_workflow()`
2. Review cross-validation results
3. Optimize ensemble weights
4. Fine-tune position sizing parameters
5. Generate submission file